In [16]:
from transformers import AutoModel
from tqdm import tqdm
import numpy as np
import torch
import json
import os

In [17]:
models_default = [
    ("bert-base-cased", ["mean"], 13, 
         [
            ("val_results_stsb", "test_results_stsb", "BERT$_{base}$"),
         ]
    ),
    ("google-electra-base-discriminator", ["mean"], 13, 
         [
            ("val_results_stsb","test_results_stsb", "ELECTRA$_{D\:base}$"),
         ]
    ),
    ("google-electra-base-generator", ["mean"], 13, 
         [
            ("val_results_stsb","test_results_stsb", "ELECTRA$_{G\:base}$"),
         ]
    )
]

models_wordsim = [
    ("bert-base-cased", ["mean"], 13, 
         [
            ("val_resultsword_similarity_stsb","test_resultsword_similarity_stsb", "BERT$_{base}$"),
         ]
    ),
    ("google-electra-base-discriminator", ["mean"], 13, 
         [
            ("val_resultsword_similarity_stsb","test_resultsword_similarity_stsb", "ELECTRA$_{D\:base}$"),
         ]
    ),
    ("google-electra-base-generator", ["mean"], 13, 
         [
            ("val_resultsword_similarity_stsb","test_resultsword_similarity_stsb", "ELECTRA$_{G\:base}$"),
         ]
    )
]


models_mlm = [
    ("bert-base-cased", ["mean"], 13, 
         [
            ("val_results_bert-base-cased_model_epoch_9_mlm_stsb", "test_results_bert-base-cased_model_epoch_9_mlm_stsb", "BERT$_{base}$"),
         ]
    ),
    ("google-electra-base-discriminator", ["mean"], 13, 
         [
            ("val_results_google-electra-base-discriminator_model_epoch_9_mlm_stsb", "test_results_google-electra-base-discriminator_model_epoch_9_mlm_stsb", "ELECTRA$_{D\:base}$"),
         ]
    ),
    ("google-electra-base-generator", ["mean"], 13, 
         [
            ("val_results_google-electra-base-generator_model_epoch_9_mlm_stsb", "test_results_google-electra-base-generator_model_epoch_9_mlm_stsb", "ELECTRA$_{G\:base}$ "),
         ]
    )
]


In [18]:
def generate_rows(models):
    for x in models:
        m = x[0]
        pool = x[1]
        c = x[2]
        json_names_labels = x[3]
        for p in pool:
            for (val_name, name, label) in json_names_labels:
                spearman, pearson = [], []
                std_spearman, std_pearson = [], []
                
                val_spearman, val_pearson = [], []
                for i in range(c):
                    res = json.load(open(f"../output/{m}/{p}/{i}_to_{i+1}/{name}.json"))
                    
                    std_1 = np.array(res["stdev_cosine_spearman_test"]) * 100
                    mean_1 = res["mean_cosine_spearman_test"] * 100
                    std_spearman.append((mean_1-std_1, mean_1+std_1))
                    spearman.append(mean_1)

                    std_2 = np.array(res["stdev_cosine_pearson_test"]) * 100
                    mean_2 = res["mean_cosine_pearson_test"] * 100
                    std_pearson.append((mean_2-std_2, mean_2+std_2))
                    pearson.append(mean_2)
                    
                    res_val = json.load(open(f"../output/{m}/{p}/{i}_to_{i+1}/{val_name}.json"))
                    val_spearman.append(res_val["mean_cosine_spearman_val"] * 100)
                    val_pearson.append(res_val["mean_cosine_pearson_val"] * 100)
                    
                argmax = np.argmax(val_spearman)
                print("{} & {} & {:.2f}/{:.2f} & {:.2f}/{:.2f} \\\\"
                    .format(
                        "}}".join("{\\text{".join(label.split("{")).split("}")), 
                        argmax, 
                        val_spearman[argmax], 
                        val_pearson[argmax], 
                        spearman[argmax], 
                        pearson[argmax]
                    )
                )
                test_s.append(spearman[argmax])
                test_p.append(pearson[argmax])
                val_s.append(val_spearman[argmax])
                val_p.append(val_pearson[argmax])
                
best = ""
                
test_s, test_p, val_s, val_p = [], [], [], []
for m in zip(models_default):
    generate_rows(m)
best += "\n{} {} {} {} \n".format(np.argmax(test_s), np.argmax(test_p), np.argmax(val_s), np.argmax(val_p))
print("\\midrule")


test_s, test_p, val_s, val_p = [], [], [], []
for m in zip(models_wordsim):
    generate_rows(m)    
best += "{} {} {} {} \n".format(np.argmax(test_s), np.argmax(test_p), np.argmax(val_s), np.argmax(val_p))
print("\\midrule")

test_s, test_p, val_s, val_p = [], [], [], []
for m in zip(models_mlm):
    generate_rows(m)    
best += "{} {} {} {}\n".format(np.argmax(test_s), np.argmax(test_p), np.argmax(val_s), np.argmax(val_p))

BERT$_{\text{base}}$ & 12 & 86.07/85.98 & 82.74/83.03 \\
ELECTRA$_{\text{D\:base}}$ & 3 & 82.15/82.20 & 75.29/76.96 \\
ELECTRA$_{\text{G\:base}}$ & 12 & 86.62/86.38 & 82.57/82.50 \\
\midrule
BERT$_{\text{base}}$ & 12 & 85.85/85.84 & 83.77/84.08 \\
ELECTRA$_{\text{D\:base}}$ & 3 & 82.66/82.56 & 76.71/77.31 \\
ELECTRA$_{\text{G\:base}}$ & 12 & 86.67/86.39 & 82.85/82.91 \\
\midrule
BERT$_{\text{base}}$ & 12 & 85.91/85.71 & 82.66/82.64 \\
ELECTRA$_{\text{D\:base}}$ & 7 & 84.07/83.78 & 79.90/79.92 \\
ELECTRA$_{\text{G\:base}}$  & 11 & 85.58/85.22 & 80.97/80.85 \\


In [20]:
models = [
    [
        ("google-bert_uncased_L-2_H-128_A-2", 3,"BERT$_{tiny}$"),
        ("google-bert_uncased_L-4_H-256_A-4", 5,"BERT$_{mini}$"),
        ("google-bert_uncased_L-4_H-512_A-8", 5,"BERT$_{small}$"),
        ("google-bert_uncased_L-8_H-512_A-8", 9, "BERT$_{medium}$"),
        ("bert-base-cased", 13, "BERT$_{base}$"),
        ("bert-large-cased", 25, "BERT$_{large}$"),
    ],
    [
        ("google-electra-small-discriminator", 13, "ELECTRA$_{D\:small}$"),
        ("google-electra-base-discriminator", 13, "ELECTRA$_{D\:base}$"),
        ("google-electra-large-discriminator", 25, "ELECTRA$_{D\:large}$"),
    ],
    [
        ("google-electra-small-generator", 13, "ELECTRA$_{G\:small}$"),
        ("google-electra-base-generator", 13, "ELECTRA$_{G\:base}$"),
        ("google-electra-large-generator", 25, "ELECTRA$_{G\:large}$"),

    ]
]

def count_params(model, i):
    x = torch.tensor(0.)
    for p in model.embeddings.parameters():
        x += torch.prod(torch.tensor(p.shape))
    if i > 0:
        for p in model.encoder.layer[:i].parameters():
            x += torch.prod(torch.tensor(p.shape))
    return x/10**6

all_ = []
all_scores_params_names = []
for f in models:
   
    for m in tqdm(f):
        val_means, means = [], []
        val_pearson_means, pearson_means = [], []
        
        name = m[0] if "google" not in m[0] else m[0].replace("google-", "google/")
        try:
            model = AutoModel.from_pretrained(name,add_pooling_layer=False)
        except:
            model = AutoModel.from_pretrained(name)
        
        for i in range(m[1]):
            params = count_params(model, i)
            
            res = json.load(open(f"../output/{m[0]}/mean/{i}_to_{i+1}/test_results_stsb.json"))
            means.append([res["mean_cosine_spearman_test"]*100, params])
            pearson_means.append(res["mean_cosine_pearson_test"]*100)
            
            res = json.load(open(f"../output/{m[0]}/mean/{i}_to_{i+1}/val_results_stsb.json"))
            val_means.append(res["mean_cosine_spearman_val"] * 100)
            val_pearson_means.append(res["mean_cosine_pearson_val"] * 100)

        tmp_means = means[:]
        argmax = np.argmax(val_means)
        s, num_param = np.array(means)[argmax]
        p = pearson_means[argmax]
        
        val_s = val_means[argmax]
        val_p = val_pearson_means[argmax]
        
        all_scores_params_names.append([s, num_param, val_s, p, val_p, argmax, m[2]])
        if s != tmp_means[-1][0]:
            all_scores_params_names.append(
                [
                    tmp_means[-1][0], 
                    tmp_means[-1][1],
                    val_means[-1],
                    pearson_means[-1],
                    val_pearson_means[-1],
                    m[1]-1,
                    "\:last}".join(m[2].split("}"))]
            )
        
argm_score = np.argmax([p[0] for p in all_scores_params_names])
argm_param = np.argmin([p[1] for p in all_scores_params_names])
argm_val_s = np.argmax([p[2] for p in all_scores_params_names])
argm_p = np.argmax([p[3] for p in all_scores_params_names])
argm_val_p = np.argmax([p[4] for p in all_scores_params_names])

print(
    " max spearman: ", all_scores_params_names[argm_score][-1], "\n",
    "min params: ", all_scores_params_names[argm_param][-1], "\n",
    "max val spearman: ", all_scores_params_names[argm_val_s][-1],"\n", 
    "pearson at max val spearman: ", all_scores_params_names[argm_p][-1],"\n", 
    "pearson val at max val spearman: ", all_scores_params_names[argm_val_p][-1],"\n", 
    "\n"
)

for p in all_scores_params_names:
    rounded = np.round(np.array(p[:-1]).astype(np.float32), decimals=2)
    #print(rounded)
    print(
        "}}".join("{\\text{".join(p[-1].split("{")).split("}")) + " & " +
        str(int(rounded[5])) + " & " +
        "{:.2f} & ".format(rounded[1]) +
        "{:.2f}/{:.2f} & ".format(rounded[2], rounded[4]) +
        "{:.2f}/{:.2f} ".format(rounded[0], rounded[3]) +
        "\\\\"
    )

        

100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  1.91it/s]

 max spearman:  BERT$_{large}$ 
 min params:  BERT$_{tiny}$ 
 max val spearman:  BERT$_{large}$ 
 pearson at max val spearman:  BERT$_{large}$ 
 pearson val at max val spearman:  BERT$_{large}$ 
 

BERT$_{\text{tiny}}$ & 2 & 4.37 & 78.20/77.57 & 69.80/70.64 \\
BERT$_{\text{mini}}$ & 4 & 11.10 & 83.06/82.42 & 75.55/76.28 \\
BERT$_{\text{small}}$ & 4 & 28.50 & 85.25/85.09 & 79.13/79.56 \\
BERT$_{\text{medium}}$ & 8 & 41.11 & 85.74/85.46 & 80.74/81.02 \\
BERT$_{\text{base}}$ & 12 & 107.72 & 86.07/85.98 & 82.74/83.03 \\
BERT$_{\text{large}}$ & 24 & 332.53 & 88.33/88.31 & 85.47/85.68 \\
ELECTRA$_{\text{D\:small}}$ & 1 & 4.76 & 79.74/79.27 & 68.88/69.64 \\
ELECTRA$_{\text{D\:small\:last}}$ & 12 & 13.45 & 73.98/73.14 & 66.72/67.27 \\
ELECTRA$_{\text{D\:base}}$ & 3 & 45.10 & 82.15/82.20 & 75.29/76.96 \\
ELECTRA$_{\text{D\:base\:last}}$ & 12 & 108.89 & 72.41/71.62 & 66.82/67.23 \\
ELECTRA$_{\text{D\:large}}$ & 12 & 182.94 & 84.74/84.88 & 80.90/81.15 \\
ELECTRA$_{\text{D\:large\:last}}$ & 24 & 3